In [31]:
import ibmdata
import numpy as np
import pandas as pd

%load_ext memory_profiler
from memory_profiler import profile

In [29]:
def timeit(method):
    @wraps(method)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = method(*args, **kwargs)
        end_time = time.time()
        print(f"{method.__name__} => {(end_time-start_time)} s")

        return result

    return wrapper

QUERY = f"""
        SELECT LEFT(lot_Id,5) AS lot_id, family_Code, MIN(Last_test_date) AS date, tp.parm_Label as parameter,
        AVG(weighted_Mean) as mean 
        FROM DMIW.PTileWaferFact ptwf
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testparmkey = ptwf.testparmkey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
        WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' 
        AND weighted_Mean is not null AND abs(weighted_Mean) < 1e25 
        AND ( tp.parm_Label LIKE '{PARM}' )
        AND family_Code IN '{fc}'
        GROUP BY LEFT(lot_Id,5), family_Code, tp.parm_Label
        ORDER BY MIN(Last_test_date), LEFT(lot_Id,5)
        """
        df = ibmdata.isdw.query(QUERY)
        df['mean'] = df['mean'].astype(float).round(2)
        return df

SELECT lot_Id, last_fab_lot_id, wafer_Id, family_Code, lot_Grade, date, level, testedWafer.calcdefs, testProgramName as step, last_Eqp_Id as tool, parm_Label as parameter, 
	 lower_Spec_Limit as lsl, target, upper_Spec_Limit as usl, units, display_Scale, critical_Area, 
	 1 as lotCount, 1 as waferCount, 
	 sample_Size - screened - n_Censored_Low - n_Censored_High as n, weighted_Mean as mean, 
	 case when (min <> max) then standard_Deviation else 0.0 end as stdDev, 
	 min, ptile05, ptile10, ptile25, ptile50, ptile75, ptile90, ptile95, max, 
	 yield, censor_Yield, 
	 testParm.testparmkey as testparmkey, 
 testedWafer.testedWaferKey as dmiwKey 
 from DMIW.PTileWaferFact pTileWaferFact, 
	 DMIW_SYSTEMS.TestParm testParm, 
	 (select testedWaferKey, lot_Id, last_fab_lot_id, wafer_Id, family_Code, cur_Lot_Grade as lot_Grade, last_TestTimeStamp as date, level, '' as calcdefs,  last_Eqp_Id 
	 from DMIW_SYSTEMS.TestedWafer testedWafer 
	 where Last_test_date >= (current date - 15 days) and Tech_id = '7HPP' and Family_code = 'Q6' and Level = 'M10' 
	 ) testedWafer 
 where testedWafer.testedWaferKey = pTileWaferFact.testedWaferKey and 
	 testParm.testParmKey = pTileWaferFact.testParmKey 
	 and weighted_Mean is not null and abs(weighted_Mean) < 1e25 
 order by lot_Id, wafer_Id, parm_Label, level 
  with ur 
;


In [6]:
m1parms = pd.read_csv('./Parms/M1parms.csv', header=None)
fixed_parms = []
for p in list(m1parms[0]):
    fixed_parms.append(p+"_M1")
m1parms[0] = fixed_parms

In [8]:
fixed_parms

['LKG_N+PW_M1',
 'ISO_N+N+_STI_M1',
 'ISO_N+NW_(pSP_RX2W_27_wide)_M1',
 'LKG_P+NW_M1',
 'ISO_P+P+_STI_M1',
 'ISO_P+PW_(pSP_RX2W_27_wide)_M1',
 'Short_CA_T2T_(pSP_T2T_36)_M1',
 'Short_CB_T2T_(cSP_T2T_33_CB_78*18)_M1',
 'Short_CA2CB_N_T2S_(CA/CB_31.5)_M1',
 'Short_PC2CA_N_on_FLD_MA_-4_M1',
 'Short_PC2CA_N_on_FLD_MA_0_M1',
 'Short_PC2CA_N_on_FLD_MA_4_M1',
 'Short_PC2CA_P_on_FLD_MA_-4_M1',
 'Short_PC2CA_P_on_FLD_MA_0_M1',
 'Short_PC2CA_P_on_FLD_MA_4_M1',
 'CTPB_150/80_N_1PB_CUT_M1',
 'CTPB_150/80_P_1PB_CUT_M1',
 'OPEN_PCCBM1_N_2A_D01_M1',
 'OPEN_PCCBM1_N_2A_D02_M1',
 'OPEN_PCCBM1_N_2A_D03_M1',
 'OPEN_PCCBM1_N_2A_D04_M1',
 'OPEN_PCCBM1_N_2A_D05_M1',
 'OPEN_PCCBM1_N_2A_D06_M1',
 'OPEN_PCCBM1_N_2A_D07_M1',
 'OPEN_PCCBM1_N_2A_D08_M1',
 'GRV_CT_CUT_N_long_CT_M1',
 'GRV_CT_CUT_P_long_CT_M1',
 'QSTC_CMJ_I_PC_Short_075_M1',
 'PC_RES_LVTN_M1',
 'PC_RES_sLVTN_M1',
 'PC_RES_LVTP_M1',
 'PC_RES_sLVTP_M1',
 'Short_PC2PC_N_no_CA_Epi_RX_M1',
 'Short_PC2PC_N_no_CA_M1',
 'Short_PC2PC_P_no_CA_Epi_RX_M1',
 'S

In [19]:
%%time
QUERY = f"""
        SELECT LEFT(lot_Id,5) AS lot_id, family_Code, MIN(Last_test_date) AS date, tp.parm_Label as parameter,
        AVG(weighted_Mean) as mean 
        FROM DMIW.PTileWaferFact ptwf
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testparmkey = ptwf.testparmkey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
        WHERE Last_test_date >= (current date - {365} days)
        AND Tech_id = '7HPP' 
        AND weighted_Mean is not null AND abs(weighted_Mean) < 1e25 
        AND Family_code IN ('Q6', 'X2') 
        GROUP BY LEFT(lot_Id,5), family_Code, tp.parm_Label
        ORDER BY MIN(Last_test_date), LEFT(lot_Id,5)
        """
%memit df = ibmdata.isdw.query(QUERY)
df

peak memory: 3165.58 MiB, increment: 2381.06 MiB
CPU times: user 36.9 s, sys: 5.48 s, total: 42.4 s
Wall time: 16min 19s


,lot_id,family_code,date,parameter,mean
0,A1B2F,Q6,2021-07-19,ASB010S90V0I0P0PALR0g2p_abt_Obyp10d2_abst_mfgf...,0.008249
1,A1B2F,Q6,2021-07-19,ASP10A100RPSTDEFAL36g2p_abt_Oplp10d2_abst_rstr...,0.090560
2,A1B2F,Q6,2021-07-19,ASP10A100RPSTNRWAL36g2p_abt_Oplp10d2_abst_rstr...,0.071468
3,A1B2F,Q6,2021-07-19,ASP10AB10K0N0D0XAL33g2p_abt_Oplp10d2_abst_rdch...,0.145385
4,A1B2F,Q6,2021-07-19,ASP10AB10L0N0D0XAL33g2p_abt_Oplp10d2_abst_rdre...,0.134939
...,...,...,...,...,...
4834862,A2C01,X2,2022-07-19,ASP500P05M0M0D0_ALR0_Pmax,311.917998
4834863,A2C01,X2,2022-07-19,ASP500P05M0M0D0_ALR0_Rload,0.000000
4834864,A2C01,X2,2022-07-19,ASP500P05M0M0D0_ALR0g2p_pre_za2_abst_mfgfullxx...,6.321389
4834865,A2C01,X2,2022-07-19,ASP500P05REDUNDTALR0g2p_pre_za2_abst_mfgfullxx...,0.147222


In [53]:
df_LT = df[['_LT' in param for param in df['parameter']]]

In [54]:
len(df_LT['parameter'].unique())

1

In [55]:
df_LT['parameter'].unique()

array(['PCHK_0.1V_01_LT'], dtype=object)

In [56]:
mfparms = pd.read_csv('./Parms/MFparms.csv', header=None)
fixed_parms = []
for p in list(mfparms[0]):
    fixed_parms.append(p+"_LT")
mfparms[0] = fixed_parms

In [57]:
fixed_parms[0]

'PCHK_0.1V_01_LT'

In [33]:
df2 = (df_LT[df_LT.parameter == 'PCHK_0.1V_01_LT' and df_LT.family_code == 'X2']).reset_index(drop=True)

NameError: name 'df_LT' is not defined

In [59]:
df2

,lot_id,family_code,date,parameter,mean
0,AZCYZ,X2,2022-02-04,PCHK_0.1V_01_LT,1.535291
1,AZCXV,X2,2022-02-07,PCHK_0.1V_01_LT,2.185297
2,AZCZF,X2,2022-02-25,PCHK_0.1V_01_LT,1.812690
3,AZCZK,X2,2022-02-28,PCHK_0.1V_01_LT,1.703438
4,AZCZV,X2,2022-03-06,PCHK_0.1V_01_LT,1.632063
...,...,...,...,...,...
70,A2B0Y,X2,2022-07-11,PCHK_0.1V_01_LT,13230.020635
71,A2B3P,X2,2022-07-11,PCHK_0.1V_01_LT,15.034094
72,A29UA,X2,2022-07-12,PCHK_0.1V_01_LT,1.887783
73,A2B6K,X2,2022-07-12,PCHK_0.1V_01_LT,1.864471


In [60]:
%%time
QUERY = f"""
        SELECT LEFT(lot_Id,5) AS lot_id, family_Code, MIN(Last_test_date) AS date, tp.parm_Label as parameter,
        AVG(weighted_Mean) as mean 
        FROM DMIW.PTileWaferFact ptwf
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testparmkey = ptwf.testparmkey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
        WHERE Last_test_date >= (current date - {365} days)
        AND Tech_id = '7HPP' 
        AND weighted_Mean is not null AND abs(weighted_Mean) < 1e25 
        AND ( tp.parm_Label LIKE '{'PCHK_0.1V_01_LT'}' )
        AND family_Code IN '{'X2'}'
        GROUP BY LEFT(lot_Id,5), family_Code, tp.parm_Label
        ORDER BY MIN(Last_test_date), LEFT(lot_Id,5)
        """
%memit df = ibmdata.isdw.query(QUERY)
df

peak memory: 2083.75 MiB, increment: 0.27 MiB
CPU times: user 898 ms, sys: 185 ms, total: 1.08 s
Wall time: 9.01 s


,lot_id,family_code,date,parameter,mean
0,AZCYZ,X2,2022-02-04,PCHK_0.1V_01_LT,1.535291
1,AZCXV,X2,2022-02-07,PCHK_0.1V_01_LT,2.185297
2,AZCZF,X2,2022-02-25,PCHK_0.1V_01_LT,1.812690
3,AZCZK,X2,2022-02-28,PCHK_0.1V_01_LT,1.703438
4,AZCZV,X2,2022-03-06,PCHK_0.1V_01_LT,1.632063
...,...,...,...,...,...
70,A2B0Y,X2,2022-07-11,PCHK_0.1V_01_LT,13230.020635
71,A2B3P,X2,2022-07-11,PCHK_0.1V_01_LT,15.034094
72,A29UA,X2,2022-07-12,PCHK_0.1V_01_LT,1.887783
73,A2B6K,X2,2022-07-12,PCHK_0.1V_01_LT,1.864471


In [61]:
df.compare(df2)

Empty DataFrame
Columns: []
Index: []

In [ ]:
# cpd class

# initialize output df

# Pull all data

# save to cpd object

